In [1]:
########################################################
#NOTES
########################################################
#print the count that each unique set of binary values occur in the dataset
#df2.value_counts()

#so .iloc[[0]] gets the first row of the series
#.index https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html
#toList() converts the index to a list with a tuple of the unique set of binary values
#the first square brackets after the tolist selects the only item in the list
#the second square brackets selects some item at that index in the tuple
#df2.value_counts().iloc[[6]].index.tolist()[0][0]

#gets the count that the first unique set of binary values occur in the dataset
#df2.value_counts().iloc[0]

#number of items in dictionary = len(a)
#select one instance from the dictionary = a[0]
#each instance in dictionary is a tuple = type(a[0])
#_______________________________________________________
########################################################

In [2]:
########################################################
#IMPORTS
########################################################
#importing train_test_split to split the data with 
from sklearn.model_selection import train_test_split
#importing pandas to manipulate data with
import pandas as pd
#importing resample to upsample any data that's in the minority
from sklearn.utils import resample
#https://stackoverflow.com/questions/12986272/how-do-i-compute-all-possibilities-for-an-array-of-numbers-bits-in-python-or-a
from itertools import product
#from unique_combinations import subsets
import itertools
import math
#_______________________________________________________
########################################################

In [3]:
########################################################
#ALL UNIQUE N BITS GENERATION
########################################################
def allUniqueNBitsGen(df2):
    #https://stackoverflow.com/questions/5036700/how-can-you-dynamically-create-variables
    #blank dictionary to store all binary combinations
    a = {}
    
    #countI represents the number of unique binary combinations generated
    countI = 0
    
    #https://stackoverflow.com/questions/12986272/how-do-i-compute-all-possibilities-for-an-array-of-numbers-bits-in-python-or-a
    #if product is set [1,0] all binary combinations will be sorted from all ones to all zeros
    #repeat is the number of bits/binary numbers
    #repeat is set to the number of columns because 2^numberOfColumns is the number of unique combinations of binary numbers
    for i in product([1,0], repeat=len(df2.columns)):
        
        #dynamically creating variables to store every tuple binary combination
        a[countI] = i
        
        #countI is iterated here, so that the first a[countI] is where countI = 0
        countI = countI + 1
        
    return a
#_______________________________________________________
########################################################

In [4]:
########################################################
#CREATE NEW BLANK DATAFRAME WITH COLUMNS FROM DATAFRAME
########################################################
def newValueCountsFrame(df2):
    #basically trying to re-create value counts (pandas function), but I want to include all columns (the ones that don't have a count)
    #creating a new empty list to add all columns from original data frame + value count
    df2_value_counts_columns = []
    
    #need to add all column names to new data frame
    #adding each column name from the data frame to a list
    for i in range(len(df2.columns)):
        
        df2_value_counts_columns.append(df2.columns.tolist()[i])
        
    #adding value count to new data frame columns
    df2_value_counts_columns.append("Value Count")
    
    #creating new value counts dataframe with columns collected
    df2_value_counts = pd.DataFrame(columns = df2_value_counts_columns)
    
    return df2_value_counts
#_______________________________________________________
########################################################

In [5]:
########################################################
#FILL BLANK DATAFRAME WITH VALUE COUNTS AND INSTANCES
########################################################
#NOTE: (a [Post Filling], df2 [Post Filling], df2_value_counts [Post Filling])
def valueCounts(a, df2, df2_value_counts, n, listLT, listLF, listLC):
    
    #for all the unique bits instances
    #len(a) in outer loop because we're nested for looping between the 'a' dictionary and the df2.value_counts
    for i in range(len(a)):
        
        #make sure to reset the flag after each iteration
        #flag holds whether or not the actual value count for the instance was greater than 0
        flag = False
        
        #this list is meant to hold the instance for the new value counts dataframe
        #make sure to clear the row list each iteration
        #need to initially set this to this because it's a list of a list
        row = list(a[i])
        
        #for the number of bits in the current dictionary item
        for k in range(len(df2.value_counts())):
            
            #tuples can be used like: is (1,0,0) == (1,0,0) and it will return true
            #and (1,0,0) == (0,1,0) will return false
            if df2.value_counts().iloc[[k]].index.tolist()[0] == a[i]:
                
                #if these match, value count for actual dataframe instance is greater than 0
                flag = True
                
                #append the value count of the dataframe to the new value counts
                #keeps me from needing to figure out how to actually count the number of times it occurs
                row.append(df2.value_counts().iloc[k])
                
        #if the value count was not greater than 0
        if flag == False:
            
            #append that to the row
            row.append(0)
            
        #add instance to new value counts
        df2_value_counts.loc[i] = row
        
    #TESTING
    #print(df2_value_counts)
    rule(df2, df2_value_counts, n, listLT, listLF, listLC)
########################################################

In [6]:
########################################################
#RULE CREATION PART
########################################################
def rule(df2, df2_value_counts, n, listLT, listLF, listLC):
    
    #list1 stores all of the rules for the data frame
    list1 = []
    list2 = []
    
    #PRNR stores the positive relationships in the dataframe and negative relationships from the data frame
    #rainbow single colors in the order in which they occur, the size of PRNR is half of the size of the 
    #value counts of the dataframe
    PRNR = {}
    
    numberOfInstances = len(df2)
    
    maxPRNR = 0
    indexMaxPRNR = 0
    maxPosPredVC = 0
    indexMaxPosPredVC = 0
    maxNegPredVC = 0
    indexMaxNegPredVC = 0
    
    #number of unique value counts for a given data frame is 2 to the n where n is the number of columns
    for i in range(pow(2, n)):
        
        #POSITIVE PREDICTION HEURISTIC INDEXING LOGIC
        #in the value counts dataframe, all even rows contain a positive label
        if i % 2 == 0:
            ########################################################
            #print("POSITIVE!")
            #print(df2_value_counts.iloc[[i]])
            #print("--------------------------")
            #print()
            ########################################################
            
            if maxPosPredVC < df2_value_counts.iloc[[i]]["Value Count"].values[0]:
                maxPosPredVC = df2_value_counts.iloc[[i]]["Value Count"].values[0]
                indexMaxPosPredVC = i
        
        #NEGATIVE PREDICTION HEURISTIC INDEXING LOGIC
        #in the value counts dataframe, all odd rows contain a negative label
        if i % 2 == 1:
            ########################################################
            #print("NEGATIVE!")
            #print(df2_value_counts.iloc[[i]])
            #print("--------------------------")
            #print()
            ########################################################
            
            if maxNegPredVC < df2_value_counts.iloc[[i]]["Value Count"].values[0]:
                maxNegPredVC = df2_value_counts.iloc[[i]]["Value Count"].values[0]
                indexMaxNegPredVC = i
        
        #COMBINATORIAL PREDICTION HEURISTIC INDEXING LOGIC
        #in the value counts dataframe, all rows ((pow(2, n) - 1) - index of row < (pow(2,n/2))) away from
        #any given index of a row less than (pow(2,n/2)) is the inverse of that row
        if i < (pow(2, n)/2):
            ########################################################
            #print("COMBINATORIAL!")
            #print(df2_value_counts.iloc[[i]])
            #print(df2_value_counts.iloc[[((pow(2, n) - 1) - i)]])
            #print("--------------------------")
            #print()
            ########################################################
            
            #PRNR logic AKA POS if X, NEG if -X, or POS if -X, NEG if X
            #for brevity's sake
            k = df2_value_counts.iloc[[i]]["Value Count"].values[0]
            #need to collect the corresponding value column for k by the definition of P.R and N.R
            #this can be accomplished with the formula below i.e. formula gets second number (0,7)(1,6)(2,5)(3,4) 
            #where n = 3 
            z = ((pow(2, n) - 1) - i)
            x = df2_value_counts.iloc[[z]]["Value Count"].values[0]
            PRNR[i] = k + x
            #the idea here is that, x and k combined need to represent more than 50% of the dataset to mean anything
            #worthy of note
            if x/numberOfInstances > 0.25 and k/numberOfInstances > 0.25:
                if PRNR[i] > maxPRNR:
                    maxPRNR = PRNR[i]
                    indexMaxPRNR = i
                
            list2.append([i, z])
            
    #if there exists no PRNR relationship > 50% for the dataframe
    if maxPRNR == 0:
        
        #if there exists a positive prediction value count for the dataframe
        if maxPosPredVC != 0:

            #subtract 1 for computer science numbering and another 1 to get past the value counts
            labelName = df2_value_counts.iloc[[indexMaxPosPredVC]].columns[len(df2_value_counts.iloc[0]) - 2]

            first = df2_value_counts.iloc[[indexMaxPosPredVC]]
            combinedRule = ""

            combinedRule = combinedRule + 'The heuristic "If '

            for z in range(len(df2_value_counts.columns) - 2):
                columnName = first.columns[z]
                columnNameValAtIndex = first[columnName].values[0]

                if((len(df2_value_counts.columns) - 2) >= 3 and z == len(df2_value_counts.columns) - 3):
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + "and " + str(columnName) + " is true; "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + "and " + str(columnName) + " is false; "
                elif (len(df2_value_counts.columns) - 2) == 2 and z == 0:
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + str(columnName) + " is true and "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + str(columnName) + " is false and "
                else:
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + str(columnName) + " is true, "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + str(columnName) + " is false, "

            labelNameValAtIndex = first[labelName].values[0]

            instanceValueCounts = first["Value Count"].values[0]

            if labelNameValAtIndex == 1:
                combinedRule = combinedRule + "then " + str(labelName) + ' is true." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances in the sub-dataset, incorrectly labeling " + str(numberOfInstances - instanceValueCounts) + " of the " + str(numberOfInstances) + "."
                #print()
                #print(combinedRule)
                list1 = labelName, df2_value_counts.columns, combinedRule, (instanceValueCounts/numberOfInstances) * 100, df2_value_counts, df2
                #print()

            if labelNameValAtIndex == 0:
                combinedRule = combinedRule + "then " + str(labelName) + ' is false." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances in the sub-dataset, incorrectly labeling " + str(numberOfInstances - instanceValueCounts) + " of the " + str(numberOfInstances) + "."
                #print()
                #print(combinedRule)
                list1 = labelName, df2_value_counts.columns, combinedRule, (instanceValueCounts/numberOfInstances) * 100, df2_value_counts, df2
                #print()

            listLT.append(list1)

        #if there exists a largest negative prediction value count for the dataframe
        if maxNegPredVC != 0:
            labelName = df2_value_counts.iloc[[indexMaxNegPredVC]].columns[len(df2_value_counts.iloc[0]) - 2]

            first = df2_value_counts.iloc[[indexMaxNegPredVC]]
            combinedRule = ""

            combinedRule = combinedRule + 'The heuristic "If '

            for z in range(len(df2_value_counts.columns) - 2):
                columnName = first.columns[z]
                columnNameValAtIndex = first[columnName].values[0]

                if((len(df2_value_counts.columns) - 2) >= 3 and z == len(df2_value_counts.columns) - 3):
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + "and " + str(columnName) + " is true; "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + "and " + str(columnName) + " is false; "
                elif (len(df2_value_counts.columns) - 2) == 2 and z == 0:
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + str(columnName) + " is true and "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + str(columnName) + " is false and "
                else:
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + str(columnName) + " is true, "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + str(columnName) + " is false, "

            labelNameValAtIndex = first[labelName].values[0]

            instanceValueCounts = first["Value Count"].values[0]

            if labelNameValAtIndex == 1:
                combinedRule = combinedRule + "then " + str(labelName) + ' is true." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances in the sub-dataset, incorrectly labeling " + str(numberOfInstances - instanceValueCounts) + " of the " + str(numberOfInstances) + "."
                #print()
                #print(combinedRule)
                list1 = labelName, df2_value_counts.columns, combinedRule, (instanceValueCounts/numberOfInstances) * 100, df2_value_counts, df2
                #print()

            if labelNameValAtIndex == 0:
                combinedRule = combinedRule + "then " + str(labelName) + ' is false." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances in the sub-dataset, incorrectly labeling " + str(numberOfInstances - instanceValueCounts) + " of the " + str(numberOfInstances) + "."
                #print()
                #print(combinedRule)
                list1 = labelName, df2_value_counts.columns, combinedRule, (instanceValueCounts/numberOfInstances) * 100, df2_value_counts, df2
                #print()

            listLF.append(list1)
    
    #if there exists a PRNR relationship > 50% for the dataframe
    else:

        labelName = df2_value_counts.iloc[[indexMaxPRNR]].columns[len(df2_value_counts.iloc[0]) - 2]
        
        #print(df2_value_counts.head())
        #print("-----------------------")
        #print()

        first = df2_value_counts.iloc[[list2[indexMaxPRNR][0]]]
        second = df2_value_counts.iloc[[list2[indexMaxPRNR][1]]]
        combinedRule = ""

        #print("labelName = ", labelName)

        combinedRule = combinedRule + 'The heuristic "If '

        for j in range(len(list2[indexMaxPRNR])):

            for z in range(len(df2_value_counts.columns) - 2):

                columnName = df2_value_counts.iloc[[list2[indexMaxPRNR][j]]].columns[z]
                columnNameValAtIndex = df2_value_counts.iloc[[list2[indexMaxPRNR][j]]][columnName].values[0]

                if((len(df2_value_counts.columns) - 2) >= 3 and z == len(df2_value_counts.columns) - 3):
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + "and " + str(columnName) + " is true; "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + "and " + str(columnName) + " is false; "
                elif (len(df2_value_counts.columns) - 2) == 2 and z == 0:
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + str(columnName) + " is true and "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + str(columnName) + " is false and "
                else:
                    if columnNameValAtIndex == 1:
                        combinedRule = combinedRule + str(columnName) + " is true, "
                    elif columnNameValAtIndex == 0:
                        combinedRule = combinedRule + str(columnName) + " is false, "

            labelNameValAtIndex = df2_value_counts.iloc[[list2[indexMaxPRNR][j]]][labelName].values[0]
            instanceValueCounts = df2_value_counts.iloc[[list2[indexMaxPRNR][j]]]["Value Count"].values[0]

            if j == 0:
                if labelNameValAtIndex == 1:
                    combinedRule = combinedRule + "then "+ str(labelName) + ' is true." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances, "

                if labelNameValAtIndex == 0:
                    combinedRule = combinedRule + "then " + str(labelName) + ' is false." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances, "
            else:
                if labelNameValAtIndex == 1:
                    combinedRule = combinedRule + "then " + str(labelName) + ' is true." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances"

                if labelNameValAtIndex == 0:
                    combinedRule = combinedRule + "then " + str(labelName) + ' is false." correctly labels ' + str((instanceValueCounts/numberOfInstances) * 100) + "% of the instances"

            if(j != len(list2[indexMaxPRNR]) - 1):
                combinedRule = combinedRule + 'and the heuristic "If '

            else:
                combinedRule = combinedRule + ", for a combined rule accuracy of " + str((PRNR[indexMaxPRNR]/numberOfInstances) * 100) + "%, incorrectly labeling " + str(numberOfInstances - PRNR[indexMaxPRNR]) + " of the " + str(numberOfInstances) + " instances in the sub-dataset."
                #print()
                #print(combinedRule)
                list1 = labelName, df2_value_counts.columns, combinedRule, (PRNR[indexMaxPRNR]/numberOfInstances) * 100, df2_value_counts, df2
                #print()

        listLC.append(list1)

    #TESTING
    #print("PRNR = ", PRNR)
#_______________________________________________________
########################################################

In [7]:
########################################################
#SORT AND PRINT
########################################################
#https://www.delftstack.com/howto/python/sort-list-of-lists-in-python/
def sortAndPrint(listListLists):
    from operator import itemgetter
    print()
    print("Best Heuristics for All Sub-Datasets:")
    print("_____________________________________")
    print()
    
    sortedListLists = []
    
    for i in range(len(listListLists)): #for each label iterated through
            
        sortedListLists = sorted(listListLists[i], key=itemgetter(3), reverse=True)
        print("The Best Rule for Label", sortedListLists[0][0])
        print("-------------------------")
        print(sortedListLists[0][2]) #at index 2 is the rule
        print(sortedListLists[0][4])
        print()
    

In [8]:
########################################################
#HONETSCHLAGER'S METHOD (CATEGORICAL DATAFRAME)
########################################################
from operator import itemgetter
def honetschlagersMethod2(n, df, df_initial):
    #to store variables printed from the method
    #this needs to be set way at the outset so that the variable doesn't 
    #get wiped/reset, so that we store everything output
    
    listListLists = []
    #THE NUMBER OF TOTAL COLUMNS (AFTER Y) NEEDS TO BE 
    #GREATER THAN OR EQUAL TO 2
    #n is number of unique columns
    n = n
    
    #for all columns in the initial un-dummied data set
    for (columnName, columnData) in df_initial.iteritems():
        
        print(columnName)
        parentColumnName = columnName
        
        for (columnName, columnData) in df.iteritems():
            
            #because for loop starts from beginning of df, need to make sure we have the parent's child
            if parentColumnName in columnName:
                y = df[columnName]
                y_columnName = columnName
                
                X = df

                for (columnName, columnData) in df.iteritems():
                    #check if the parent column name is present in the dummy column name
                    #this ensures we're specifying one parent column to pull our y vector or labels from
                    if parentColumnName in columnName:

                        X = X.drop(columnName, axis=1)
                        
                #list of label true rule instance details
                listLT = []
                
                #list of label false rule instance details
                listLF = []
                
                #list of combinatorial label rule instance details
                listLC = []

                #all unique combinations of columns in the dataframe
                unique_combinations_listI = list(itertools.combinations(X.columns, n))

                #for all tuples in the list of tuples
                for i in range(len(unique_combinations_listI)):

                    #need to initiate data frame with a place holder column name so we can easily insert columns into the dataframe
                    df_test = pd.DataFrame(columns = ['a'])

                    #for every item inside the current tuple except for y, which will be the last item in the tuple hence - 1
                    for j in range(len(unique_combinations_listI[0])):

                        #insert each column into our testing dataframe
                        #insert(loc to insert the column, column name, data for the column)
                        df_test.insert(j, unique_combinations_listI[i][j], df[unique_combinations_listI[i][j]])

                    #TESTING
                    #print()
                    #let the user know what label we're working on
                    #print('Label: ' + y.name)
                    #print()

                    #insert y at the end of the test dataframe
                    df_test.insert(len(df_test.columns), y.name, y)

                    #pop column a as it's no longer needed
                    df_test.pop('a')

                    #add 1 to n because in the value counts method we refer to n as though it's the number of unique
                    #columns in the testing dataframe and not just in the x vector
                    valueCounts(allUniqueNBitsGen(df_test), df_test, newValueCountsFrame(df_test), n + 1, listLT, listLF, listLC)
                
                if len(listLT) > 0:
                    listLT = sorted(listLT, key=itemgetter(3), reverse=True)
                    print('The Best Rule for "label', "'" , listLF[0][0], "'", 'is true"')
                    print("------------------------------------------------------")
                    print(listLT[0][2]) #at index 2 is the rule
                    print()
                    print("Value Counts for Sub-Dataset: ")
                    print(listLT[0][4])
                    #print(listLT)
                    print()

                if len(listLF) > 0:
                    listLF = sorted(listLF, key=itemgetter(3), reverse=True)
                    print('The Best Rule for "label', "'", listLF[0][0], "'", 'is false"')
                    print("-------------------------------------------------------")
                    print(listLF[0][2]) #at index 2 is the rule
                    print()
                    print("Value Counts for Sub-Dataset: ")
                    print(listLF[0][4])
                    print()

                if len(listLC) > 0:
                    listLC = sorted(listLC, key=itemgetter(3), reverse=True)
                    print("The Best Combinatorial Rule for Label '", listLC[0][0], "'")
                    print("----------------------------------------------------")
                    print(listLC[0][2]) #at index 2 is the rule
                    print()
                    print("Value Counts for Sub-Dataset: ")
                    print(listLC[0][4])
                    print()


            #TESTING
            #print()
            #print("------------------------------------------------")
    #sortAndPrint(listListLists)
#_______________________________________________________
########################################################

In [25]:
########################################################
#HONETSCHLAGER'S METHOD (BINARY DATAFRAME NO PARENTS)
########################################################
from operator import itemgetter
def honetschlagersMethod(n, df):
    #to store variables printed from the method
    #this needs to be set way at the outset so that the variable doesn't 
    #get wiped/reset, so that we store everything output
    
    listListLists = []
    #THE NUMBER OF TOTAL COLUMNS (AFTER Y) NEEDS TO BE 
    #GREATER THAN OR EQUAL TO 2
    #n is number of unique columns
    n = n
    
    for (columnName, columnData) in df.iteritems():
        
        #list of label true rule instance details
        listLT = []

        #list of label false rule instance details
        listLF = []

        #list of combinatorial label rule instance details
        listLC = []
        
        y = df[columnName]
        X = df.drop(columnName, axis=1)
        
        #all unique combinations of columns in the dataframe
        unique_combinations_listI = list(itertools.combinations(X.columns, n))

        #for all tuples in the list of tuples
        for i in range(len(unique_combinations_listI)):
            df_resampled = pd.DataFrame()

            #need to initiate data frame with a place holder column name so we can easily insert columns into the dataframe
            df_test = pd.DataFrame(columns = ['a'])

            #for every item inside the current tuple except for y, which will be the last item in the tuple hence - 1
            for j in range(len(unique_combinations_listI[0])):

                #insert each column into our testing dataframe
                #insert(loc to insert the column, column name, data for the column)
                df_test.insert(j, unique_combinations_listI[i][j], df[unique_combinations_listI[i][j]])

            #TESTING
            #print()
            #let the user know what label we're working on
            #print('Label: ' + y.name)
            #print()

            #insert y at the end of the test dataframe
            df_test.insert(len(df_test.columns), y.name, y)
            
            #pop column a as it's no longer needed
            df_test.pop('a')

            #UPSAMPLING
            #resample the dataframe_skip to equal the value counts of columns with 0
            #n_samples = df_dummies[skip].value_counts().max(), replace = True) #old n samples statement
            df_resampled = resample(df_test[df_test[y.name] == 0], n_samples = 200, replace = True)
            #apppend the columns with 1
            df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))
            df_resampled = df_resampled.sample(frac = 1).reset_index(drop=True)
            
            #NO UPSAMPLING
            ###df_resampled = df_test.sample(frac = 1).reset_index(drop=True)

            #add 1 to n because in the value counts method we refer to n as though it's the number of unique
            #columns in the testing dataframe and not just in the x vector
            valueCounts(allUniqueNBitsGen(df_resampled), df_resampled, newValueCountsFrame(df_resampled), n + 1, listLT, listLF, listLC)
        
        if len(listLT) > 0:
            listLT = sorted(listLT, key=itemgetter(3), reverse=True)
            print('The Best Rule for "label', "'" , listLF[0][0], "'", 'is true"')
            print("------------------------------------------------------")
            print(listLT[0][2]) #at index 2 is the rule
            print()
            print("Value Counts for Sub-Dataset: ")
            print(listLT[0][4])
            #print(listLT)
            print()

        if len(listLF) > 0:
            listLF = sorted(listLF, key=itemgetter(3), reverse=True)
            print('The Best Rule for "label', "'", listLF[0][0], "'", 'is false"')
            print("-------------------------------------------------------")
            print(listLF[0][2]) #at index 2 is the rule
            print()
            print("Value Counts for Sub-Dataset: ")
            print(listLF[0][4])
            print()

        if len(listLC) > 0:
            listLC = sorted(listLC, key=itemgetter(3), reverse=True)
            print("The Best Combinatorial Rule for Label '", listLC[0][0], "'")
            print("----------------------------------------------------")
            print(listLC[0][2]) #at index 2 is the rule
            print()
            print("Value Counts for Sub-Dataset: ")
            print(listLC[0][4])
            print()
        
        #TESTING
        #print()
        #print("------------------------------------------------")
    #sortAndPrint(listListLists)
#_______________________________________________________
########################################################

In [26]:
def main():
    
    ########################################################
    #for a dataframe that's already in binary/no parents
    ########################################################
    #I collect the non-dummy data incase it's necessary to have it
    #sample shuffles the data
    #reset_index resets the numbering on the side of the instances, to remove traces of shuffling
    #must have 2 or more columns
    df = pd.read_excel("Data\\XYZ.xlsx").sample(frac=1).reset_index(drop=True)
    df_initial = pd.DataFrame()
    
    ########################################################
    #for a dataframe that's categorical with parents
    ########################################################
    #collecting data
    #dropped survey year because it contains the same value, "2013", repeated for every instance in the data
    #dropped three digit residential zip code because it contains too many categorical values for getting dummies
    #I collect the non-dummy data incase it's necessary to have it
    ###df_initial = pd.read_csv("Data\\Patient_Characteristics_Survey__PCS___2013.csv").drop(['Survey Year', 'Three Digit Residential Zip Code'], axis=1)
    #randomize the data frame and reset the index
    ###df_initial = df_initial.sample(frac = 0.1).reset_index(drop=True)
    #drops x columns from the beginning of the dataframe
    #df_initial.drop(df_initial.columns[[range(x)]], axis = 1, inplace = True)
    #print("len(df_initial.columns) = ", len(df_initial.columns))
    ###print(df_initial.head())
    #I collect the dummy data because I've been told it proves more useful
    ###df = pd.get_dummies(df_initial)
    #print("len(df_initial) =", len(df_initial))
    
    userSelXVSize = int(input("Please enter the number of columns you'd like to have in the X-Vector: "))
    
    while (userSelXVSize > len(df.columns)):
        print()
        print("Columns must be less than", len(df.columns))
        print()
        userSelXVSize = int(input("Please enter the number of columns you'd like to have in the X-Vector: "))
        
    print()

    if len(df_initial.columns) > 0:
        #print("Dataset: ")
        #print(df_initial)
        #print()
        #print("Dummied Dataset: ")
        #print(df)
        honetschlagersMethod2(userSelXVSize, df,df_initial)
    else:
        print("Dataset: ")
        print(df)
        print()
        honetschlagersMethod(userSelXVSize, df)

In [27]:
if __name__ == '__main__':
    main()
    
    #skewed negative/positive count

Please enter the number of columns you'd like to have in the X-Vector: 1

Dataset: 
      X    Y    Z
0   1.0  1.0  1.0
1   1.0  1.0  0.0
2   NaN  NaN  NaN
3   1.0  0.0  1.0
4   NaN  NaN  NaN
5   NaN  NaN  NaN
6   NaN  NaN  NaN
7   0.0  1.0  1.0
8   1.0  0.0  1.0
9   NaN  NaN  1.0
10  1.0  1.0  0.0
11  0.0  1.0  1.0
12  0.0  1.0  0.0
13  0.0  0.0  1.0
14  NaN  NaN  NaN
15  NaN  NaN  NaN
16  1.0  1.0  1.0
17  1.0  1.0  1.0
18  0.0  0.0  1.0
19  1.0  0.0  0.0
20  0.0  0.0  1.0
21  1.0  0.0  0.0
22  0.0  1.0  0.0
23  0.0  0.0  0.0
24  1.0  1.0  0.0
25  1.0  0.0  0.0
26  0.0  1.0  0.0
27  0.0  0.0  1.0
28  NaN  NaN  NaN
29  NaN  NaN  NaN
30  1.0  1.0  1.0
31  1.0  0.0  1.0
32  0.0  0.0  1.0
33  0.0  0.0  1.0
34  NaN  NaN  NaN
35  NaN  NaN  NaN
36  1.0  1.0  1.0

The Best Rule for "label ' X ' is true"
------------------------------------------------------
The heuristic "If Z is true, then X is true." correctly labels 31.0% of the instances in the sub-dataset, incorrectly labeling 276 of th

C:\Users\Kristopher\AppData\Local\Temp\ipykernel_3652\1498903839.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))
C:\Users\Kristopher\AppData\Local\Temp\ipykernel_3652\1498903839.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))
C:\Users\Kristopher\AppData\Local\Temp\ipykernel_3652\1498903839.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))


The Best Rule for "label ' Y ' is true"
------------------------------------------------------
The heuristic "If Z is true, then Y is true." correctly labels 26.25% of the instances in the sub-dataset, incorrectly labeling 295 of the 400.

Value Counts for Sub-Dataset: 
   Z  Y  Value Count
0  1  1          105
1  1  0          143
2  0  1           95
3  0  0           57

The Best Rule for "label ' Y ' is false"
-------------------------------------------------------
The heuristic "If Z is true, then Y is false." correctly labels 35.75% of the instances in the sub-dataset, incorrectly labeling 257 of the 400.

Value Counts for Sub-Dataset: 
   Z  Y  Value Count
0  1  1          105
1  1  0          143
2  0  1           95
3  0  0           57

The Best Combinatorial Rule for Label ' Y '
----------------------------------------------------
The heuristic "If X is true, then Y is true." correctly labels 31.5% of the instances, and the heuristic "If X is false, then Y is false." correct

C:\Users\Kristopher\AppData\Local\Temp\ipykernel_3652\1498903839.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))
C:\Users\Kristopher\AppData\Local\Temp\ipykernel_3652\1498903839.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))
C:\Users\Kristopher\AppData\Local\Temp\ipykernel_3652\1498903839.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_resampled = df_resampled.append(resample(df_test[df_test[y.name] == 1], n_samples = 200, replace = True))
